In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
! pip install feature-engine

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder

In [0]:
train = pd.read_csv('/content/drive/My Drive/ML for Banking/train_fNxu4vz.csv')
test = pd.read_csv('/content/drive/My Drive/ML for Banking/test_fjtUOL8.csv')
ss = pd.read_csv('/content/drive/My Drive/ML for Banking/sample_submission_HSqiq1Q.csv')

In [0]:
train['Loan_Amount_Requested'] = train['Loan_Amount_Requested'].str.replace(',','').astype(float)
test['Loan_Amount_Requested'] = test['Loan_Amount_Requested'].str.replace(',','').astype(float)

# train.drop(['Months_Since_Deliquency'],axis=1,inplace=True)
# test.drop(['Months_Since_Deliquency'],axis=1,inplace=True)

# Feature Engineering

In [0]:
train_pre = train.copy()
test_pre = test.copy()

# num_cols = ['Loan_Amount_Requested', 'Debt_To_Income']
# int_cols = ['Inquiries_Last_6Mo','Number_Open_Accounts','Total_Accounts']
# cat_cols = ['Income_Verified','Purpose_Of_Loan','Gender']
# count = 1
# for cat_col in cat_cols:
#   for num_col in num_cols:
#     train_pre['feature'+str(count)] = train.groupby(cat_col)[num_col].transform('mean')
#     test_pre['feature'+str(count)] = test[cat_col].replace(dict(train.groupby(cat_col)[num_col].mean()))

#     train_pre['feature'+str(count+1)] = train_pre[num_col] - train_pre['feature'+str(count)]
#     test_pre['feature'+str(count+1)] = test_pre[num_col] - test_pre['feature'+str(count)]
#     count += 2

# for cat_col in cat_cols:
#   for num_col in int_cols:
#     train_pre['feature'+str(count)] = train.groupby(cat_col)[num_col].transform('median')
#     test_pre['feature'+str(count)] = test[cat_col].replace(dict(train.groupby(cat_col)[num_col].median()))

#     train_pre['feature'+str(count+1)] = train_pre[num_col] - train_pre['feature'+str(count)]
#     test_pre['feature'+str(count+1)] = test_pre[num_col] - test_pre['feature'+str(count)]
#     count += 2


In [0]:
train['Home_Owner_fill'] = train['Home_Owner'].fillna('missing')
test['Home_Owner_fill'] = test['Home_Owner'].fillna('missing')

In [0]:
test_pre['Interest_Rate'] = np.NaN

# Label Encoding
# keys = train['Length_Employed'].unique().tolist()
# values = [0,4,7,8,2,11,1,np.NaN,6,9,3,5]
# dict_map = dict(zip(keys,values))
# train_pre['Length_Employed'] = train['Length_Employed'].replace(dict_map)
# test_pre['Length_Employed'] = test['Length_Employed'].replace(dict_map)

# keys = train['Gender'].unique().tolist()
# values = [0,1]
# dict_map = dict(zip(keys,values))
# train_pre['Gender'] = train['Gender'].replace(dict_map)
# test_pre['Gender'] = test['Gender'].replace(dict_map)

# Label Encode
le = LabelEncoder()
train_pre["Length_Employed"] = train_pre["Length_Employed"].fillna('missing')
test_pre["Length_Employed"] = test_pre['Length_Employed'].fillna('missing')

train_pre["Length_Employed"] = le.fit_transform(train_pre["Length_Employed"])
test_pre["Length_Employed"] = le.transform(test_pre["Length_Employed"])

train_pre["Gender"] = le.fit_transform(train_pre["Gender"])
test_pre["Gender"] = le.transform(test_pre["Gender"])

# Label Encoding / One Hot Encoding

# Missing values
import feature_engine.missing_data_imputers as mdi
imputer = mdi.CategoricalVariableImputer(imputation_method = 'missing',variables=['Home_Owner'])
imputer.fit(train_pre)

mean_imputer = mdi.MeanMedianImputer(imputation_method='mean', variables=['Annual_Income','Length_Employed'])
mean_imputer.fit(train_pre)

# transform the data
train_pre= imputer.transform(train_pre)
test_pre= imputer.transform(test_pre)

train_pre= mean_imputer.transform(train_pre)
test_pre= mean_imputer.transform(test_pre)

from feature_engine import categorical_encoders as ce
# set up the encoder
encoder = ce.OneHotCategoricalEncoder(variables=['Home_Owner', 'Income_Verified', 'Purpose_Of_Loan'],
                                      drop_last=True)
# fit the encoder
encoder.fit(train_pre)

# transform the data
train_pre= encoder.transform(train_pre)
test_pre= encoder.transform(test_pre)

train_pre["Home_Owner"] = le.fit_transform(train["Home_Owner_fill"])
test_pre["Home_Owner"] = le.transform(test["Home_Owner_fill"])

train_pre["Income_Verified"] = le.fit_transform(train["Income_Verified"])
test_pre["Income_Verified"] = le.transform(test["Income_Verified"])

train_pre["Purpose_Of_Loan"] = le.fit_transform(train["Purpose_Of_Loan"])
test_pre["Purpose_Of_Loan"] = le.transform(test["Purpose_Of_Loan"])

In [0]:
# train_pre['Loan_Amount_Requested'] = np.log1p(train_pre['Loan_Amount_Requested'])
# test_pre['Loan_Amount_Requested'] = np.log1p(test_pre['Loan_Amount_Requested'])

# train_pre['Annual_Income'] = np.log1p(train_pre['Annual_Income'])
# test_pre['Annual_Income'] = np.log1p(test_pre['Annual_Income'])

In [0]:
# # Label Encode
# le = LabelEncoder()
# train_pre["Length_Employed"] = le.fit_transform(train_pre["Length_Employed"])
# test_pre["Length_Employed"] = le.transform(test_pre["Length_Employed"])

# train_pre["Home_Owner"] = le.fit_transform(train_pre["Home_Owner"])
# test_pre["Home_Owner"] = le.transform(test_pre["Home_Owner"])

# train_pre["Income_Verified"] = le.fit_transform(train_pre["Income_Verified"])
# test_pre["Income_Verified"] = le.transform(test_pre["Income_Verified"])

# train_pre["Purpose_Of_Loan"] = le.fit_transform(train_pre["Purpose_Of_Loan"])
# test_pre["Purpose_Of_Loan"] = le.transform(test_pre["Purpose_Of_Loan"])

# train_pre["Gender"] = le.fit_transform(train_pre["Gender"])
# test_pre["Gender"] = le.transform(test_pre["Gender"])

In [0]:
# New feature
train_pre["Number_Invalid_Acc"] = train_pre["Total_Accounts"] - train_pre["Number_Open_Accounts"]
test_pre["Number_Invalid_Acc"] = test_pre["Total_Accounts"] - test_pre["Number_Open_Accounts"]

# New feature
train_pre["Number_Years_To_Repay_Debt"] = train_pre["Loan_Amount_Requested"]/train_pre["Annual_Income"]
test_pre["Number_Years_To_Repay_Debt"] = test_pre["Loan_Amount_Requested"]/test_pre["Annual_Income"]

# # Interaction features
# interaction1=train_pre['Income_Verified']+'_'+train_pre['Home_Owner']
# interaction2=train_pre['Income_Verified']+'_'+train_pre['Purpose_Of_Loan']
# interaction3=train_pre['Income_Verified']+'_'+train_pre['Gender']

# interaction4=train_pre['Home_Owner']+'_'+train_pre['Purpose_Of_Loan']
# interaction5=train_pre['Home_Owner']+'_'+train_pre['Gender']

# interaction6=df['Purpose_Of_Loan']+'_'+df['Gender']

# train_pre['T_C']=interaction1
# train_pre['T_D']=interaction2
# train_pre['T_G']=interaction3
# train_pre['C_D']=interaction4
# train_pre['C_G']=interaction5
# train_pre['D_G']=interaction6


# LGBM with weighted F1

In [0]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import time
def weightedF1(truth, predictions):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='weighted')
    return ('weightedF1', f1, True) 

In [0]:
train_data = train_pre.drop(['Loan_ID'],axis=1).copy()
test_data = test_pre.drop(['Interest_Rate', 'Loan_ID'],axis=1).copy()

In [0]:
# drop_cols = sorted_df.index.values.tolist()[:36]
# train_data = train_pre.drop(drop_cols,axis=1).copy()
# test_data = test_pre.drop(['Interest_Rate'] + drop_cols,axis=1).copy()

In [101]:
max_iter = 5
folds = StratifiedKFold(n_splits = max_iter)
f1_scores = []
oofs = np.zeros(len(train_data))
preds_test = np.zeros(len(test_data))
preds_test1 = np.zeros(len(test_data))
preds_test2 = np.zeros(len(test_data))

train = train_data.drop(['Interest_Rate'],axis=1)
target = train_data['Interest_Rate']

feature_importance_df = pd.DataFrame()
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, target.values)):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_trn, y_trn = train.iloc[trn_idx], target.iloc[trn_idx]

    # Applying class imbalance on only train
    sm = SMOTE(random_state=42)
    X_trn, y_trn = sm.fit_resample(X_trn, y_trn)


    X_val, y_val = train.iloc[val_idx], target.iloc[val_idx]
    X_test = test_data #test[features]
    
    print(X_trn.shape)
    
    clf = LGBMClassifier( n_estimators = 5000, learning_rate = 0.01, num_leaves=200, max_depth=4, colsample_by_tree =2, reg_alpha=0.5, reg_lambda=0.5, 
                        bagging_freq=1, bagging_fraction=0.8, max_bin=50)
    _ = clf.fit(X_trn, y_trn, eval_set = [(X_trn, y_trn), (X_val, y_val)], eval_metric = weightedF1, verbose = 100, early_stopping_rounds = 200)
    
    oofs[val_idx] = clf.predict(X_val)
    preds_test += clf.predict_proba(test_data)[:,0]/max_iter
    preds_test1 += clf.predict_proba(test_data)[:,1]/max_iter
    preds_test2 += clf.predict_proba(test_data)[:,2]/max_iter

    f1_scores += [f1_score(y_val,oofs[val_idx],average='weighted')]

    print('\n Weighted F1 score of validation :', f1_score(y_val,oofs[val_idx],average='weighted'))

    fold_importance_df = pd.DataFrame({'feature': X_trn.columns.tolist(), 'importance': clf.feature_importances_})
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")


---- Fold 0 -----

(131447, 34)
Training until validation scores don't improve for 200 rounds.
[100]	training's multi_logloss: 0.99498	training's weightedF1: 0.443125	valid_1's multi_logloss: 0.996996	valid_1's weightedF1: 0.438602
[200]	training's multi_logloss: 0.968577	training's weightedF1: 0.46668	valid_1's multi_logloss: 0.972046	valid_1's weightedF1: 0.460594
[300]	training's multi_logloss: 0.953167	training's weightedF1: 0.491804	valid_1's multi_logloss: 0.957884	valid_1's weightedF1: 0.485718
[400]	training's multi_logloss: 0.943113	training's weightedF1: 0.504281	valid_1's multi_logloss: 0.949019	valid_1's weightedF1: 0.497657
[500]	training's multi_logloss: 0.936148	training's weightedF1: 0.512992	valid_1's multi_logloss: 0.943051	valid_1's weightedF1: 0.505169
[600]	training's multi_logloss: 0.930985	training's weightedF1: 0.518769	valid_1's multi_logloss: 0.938806	valid_1's weightedF1: 0.510144
[700]	training's multi_logloss: 0.926971	training's weightedF1: 0.52315	valid_

1. Using Label Encoding only gives, 
    Validation => 0.5306331541374198 and 
    PLB => 0.528834747844148
2. OHE + LE => Validation => 0.5314198938566443
PLB => 0.53029803567004
3. OHE + LE + FE(2 features added),
Validation =>
PLB =>

In [102]:
np.mean(f1_scores)

0.5320451385730955

In [0]:
ss['Interest_Rate'] = np.argmax([preds_test,preds_test1,preds_test2],axis=0) + 1

In [0]:
ss.to_csv('Submission_9_LGBM_LabelEnc_OHE_LE_FE.csv',index=False)

In [0]:
ss.head()

,Loan_ID,Interest_Rate
0,10164310,2
1,10164311,1
2,10164312,2
3,10164313,2
4,10164314,2


In [0]:
drop_cols

['Home_Owner_Mortgage',
 'Purpose_Of_Loan_other',
 'feature30',
 'Purpose_Of_Loan_car',
 'Gender',
 'Purpose_Of_Loan_moving',
 'Purpose_Of_Loan_major_purchase',
 'Home_Owner_Own',
 'Purpose_Of_Loan_medical',
 'feature19',
 'Purpose_Of_Loan_house',
 'feature21',
 'Purpose_Of_Loan_home_improvement',
 'Purpose_Of_Loan_wedding',
 'feature28',
 'Home_Owner_Other',
 'Income_Verified_not verified',
 'feature11',
 'Purpose_Of_Loan_educational',
 'feature15',
 'feature3',
 'Income_Verified_VERIFIED - income',
 'Purpose_Of_Loan_debt_consolidation',
 'feature26',
 'feature29',
 'feature27',
 'feature13',
 'feature25',
 'Purpose_Of_Loan_small_business',
 'Purpose_Of_Loan_vacation',
 'feature17',
 'feature9']